# **multimoda-rs**: Tutorial examples IVUS/OCT to centerline
## Alining from files
The first example will focus on solving the problem of aligning frames within a pullback of either IVUS or OCT images.
We will start with gated IVUS images (systole/diastole) during two different states (e.g. rest/stress).
The .csv files are expected to set up in the following style:
```text
--------------------------------------------------------------------
|      185     |       5.32     |      2.37       |        0.0     |
|      185     |       5.12     |      2.46       |        0.0     |
|      ...     |       ...      |      ...        |        ...     |
```
where the first column is the frame index the point is from, the second to forth are x-, y- and z-coordinates. The naming conventions of the files are diastolic_contours.csv, diastolic_reference_points.csv, ... (see ./data). This is in alignment with the output of the [AIVUS-CAA software](https://https://github.com/AI-in-Cardiovascular-Medicine/AIVUS-CAA).

The first goal is to align the frames within a pullback by translating their centroids to a line and rotating them towards each other minimizing Hausdorff distance of the contours and catheter contours created from the image center. The influence of the catheter (which represents the image center) on the rotation can be adjusted by the number of points passed to catheter. If no catheter should be created just pass n_points=0.

In the same function states are aligned with each other (e.g. systole to diastole) and z-distance are averaged over the two states to have comparable frame positions. If heartrate is very different (e.g. rest to stress) a resampling is performed of the lower heartrate geometry.

Load packages multimodars, and for linking the numpy package.

In [3]:
import os
from pathlib import Path
import multimodars as mm
import numpy as np

# load the provided example data
os.chdir(Path.cwd().parent / "data")

# mode full and it's meaning
rest, stress, dia, sys = mm.from_file(
    mode="full", 
    rest_input_path="ivus_rest", 
    stress_input_path="ivus_stress", 
    steps_best_rotation=300, 
    range_rotation_rad=1.57, 
    rest_output_path="output/rest", 
    stress_output_path="output/stress", 
    diastole_output_path="output/diastole", 
    systole_output_path="output/systole", 
    interpolation_steps=28, 
    image_center=(4.5, 4.5),
    radius=0.5,
    n_points=20,
)

Generating geometry for "ivus_rest"
file/path                                          loaded
ivus_rest/diastolic_contours.csv                   true
ivus_rest/diastolic_reference_points.csv           true
ivus_rest/combined_sorted_manual.csv               true
Generating geometry for "ivus_stress"
file/path                                          loaded
ivus_stress/diastolic_contours.csv                 true
ivus_stress/diastolic_reference_points.csv         true
geometry pair: diastolic geometry generated
ivus_stress/combined_sorted_manual.csv             true
geometry pair: diastolic geometry generated
Generating geometry for "ivus_rest"
file/path                                          loaded
ivus_rest/systolic_contours.csv                    true
ivus_rest/systolic_reference_points.csv            true
ivus_rest/combined_sorted_manual.csv               true
geometry pair: systolic geometry generated
Generating geometry for "ivus_stress"
file/path                                  

In [ ]:
print(rest)
print("\n")
print(rest.dia_geom)
print("\n")
print(rest.dia_geom.contours[0])

import os
import multimodars as mm
import numpy as np
os.chdir(Path.cwd().parent / "data")

rest, stress, dia, sys = mm.from_file(
    mode="full",
    rest_input_path="fixtures/rest_csv_files",
    stress_input_path="fixtures/stress_csv_files",
    rest_output_path="output/rest",
    stress_output_path="output/stress",
    diastole_output_path="output/diastole",
    systole_output_path="output/systole_output_path",
    steps_best_rotation=300,
    range_rotation_rad=1.57, 
    interpolation_steps=0,
    image_center=(4.5, 4.5),
    radius=0.5,
    n_points=20,
)

dia = mm.to_array(rest.dia_geom)
sys = mm.to_array(rest.sys_geom)
dia_stress = mm.to_array(stress.dia_geom)
sys_stress = mm.to_array(stress.sys_geom)

# np.save("fixtures/dia_rest_contours.npy", dia["contours"])
# np.save("fixtures/dia_rest_catheter.npy", dia["catheter"])
# np.save("fixtures/dia_rest_walls.npy", dia["walls"])
# np.save("fixtures/dia_rest_ref_pt.npy", dia["reference"])

# np.save("fixtures/sys_rest_contours.npy", sys["contours"])
# np.save("fixtures/sys_rest_catheter.npy", sys["catheter"])
# np.save("fixtures/sys_rest_walls.npy", sys["walls"])
# np.save("fixtures/sys_rest_ref_pt.npy", sys["reference"])

# np.save("fixtures/dia_stress_contours.npy", dia_stress["contours"])
# np.save("fixtures/dia_stress_catheter.npy", dia_stress["catheter"])
# np.save("fixtures/dia_stress_walls.npy", dia_stress["walls"])
# np.save("fixtures/dia_stress_ref_pt.npy", dia_stress["reference"])

# np.save("fixtures/sys_stress_contours.npy", sys_stress["contours"])
# np.save("fixtures/sys_stress_catheter.npy", sys_stress["catheter"])
# np.save("fixtures/sys_stress_walls.npy", sys_stress["walls"])
# np.save("fixtures/sys_stress_ref_pt.npy", sys_stress["reference"])

dia_cont = np.load("fixtures/dia_rest_contours.npy")
dia_cath = np.load("fixtures/dia_rest_catheter.npy")
dia_wall = np.load("fixtures/dia_rest_walls.npy")
dia_ref = np.load("fixtures/dia_rest_ref_pt.npy")

sys_cont = np.load("fixtures/sys_rest_contours.npy")
sys_cath = np.load("fixtures/sys_rest_catheter.npy")
sys_wall = np.load("fixtures/sys_rest_walls.npy")
sys_ref = np.load("fixtures/sys_rest_ref_pt.npy")

dia_stress_cont = np.load("fixtures/dia_stress_contours.npy")
dia_stress_cath = np.load("fixtures/dia_stress_catheter.npy")
dia_stress_wall = np.load("fixtures/dia_stress_walls.npy")
dia_stress_ref = np.load("fixtures/dia_stress_ref_pt.npy")

sys_stress_cont = np.load("fixtures/sys_stress_contours.npy")
sys_stress_cath = np.load("fixtures/sys_stress_catheter.npy")
sys_stress_wall = np.load("fixtures/sys_stress_walls.npy")
sys_stress_ref = np.load("fixtures/sys_stress_ref_pt.npy")

round_trip_rest_dia = mm.numpy_to_geometry(
    contours_arr=dia_cont,
    catheter_arr=dia_cath,
    walls_arr=dia_wall,
    reference_arr=dia_ref,
)

round_trip_rest_sys = mm.numpy_to_geometry(
    contours_arr=sys_cont,
    catheter_arr=sys_cath,
    walls_arr=sys_wall,
    reference_arr=sys_ref,
)

mm.to_obj(rest.dia_geom, "output/test")
mm.to_obj(round_trip_rest_dia, "output/test")


Diastolic Geometry(17 contours), (17 catheter), Reference Point: Point(f=385, p=0, x=3.57, y=3.67, z=24.54, aortic=false) 
Systolic Geometry(17 contours), (17 catheter), Reference Point: Point(f=319, p=0, x=3.53, y=3.46, z=20.80, aortic=false)


Geometry(17 contours, 17 walls), Catheter(17 catheter), Reference Point: Point(f=385, p=0, x=3.57, y=3.67, z=24.54, aortic=false)


Contour(id=0, points=501, centroid=(3.72, 5.25, 3.86))
Generating geometry for "fixtures/rest_csv_files"
file/path                                          loaded
fixtures/rest_csv_files/diastolic_contours.csv     true
fixtures/rest_csv_files/diastolic_reference_points.csv true
fixtures/rest_csv_files/combined_sorted_manual.csv true
geometry pair: diastolic geometry generated
Generating geometry for "fixtures/stress_csv_files"
file/path                                          loaded
fixtures/stress_csv_files/diastolic_contours.csv   true
fixtures/stress_csv_files/diastolic_reference_points.csv true
fixtures/stress

The data is now neatly ordered in pairs (e.g. diastolic and systolic geometry). Every geometry has contours for lumen and walls and a created catheter. The reference point will be used to align the geometry to the centerline. All points corresponding to a contour are also save in a contour struct.

The four pairs represent all 4 possible comparison in gated images, as for example in coronary artery anomalies (rest pulsatile lumen deformation, stress pulsatile lumen deformation, stress-induced diastolic lumen deformation and stress-induced systolic lumen deformation).

For a simple pre- and post-stenting comparison do the following:

In [ ]:
stent_comparison = mm.from_file(
    mode="doublepair",
    rest_input_path="ivus_prestent",
    stress_input_path="ivus_poststent",
    steps_best_rotation=300,
    range_rotation_rad=1.57,
    interpolation_steps=0
    )

## Alignment from array
While the alignment from file is one option, the more flexible option is to create Geometries directly from numpy array, and then perform the same operations with these Geometries.

In [ ]:
dia_cont = np.load("fixtures/dia_rest_contours.npy")
dia_cath = np.load("fixtures/dia_rest_catheter.npy")
dia_wall = np.load("fixtures/dia_rest_walls.npy")
dia_ref = np.load("fixtures/dia_rest_ref_pt.npy")

sys_cont = np.load("fixtures/sys_rest_contours.npy")
sys_cath = np.load("fixtures/sys_rest_catheter.npy")
sys_wall = np.load("fixtures/sys_rest_walls.npy")
sys_ref = np.load("fixtures/sys_rest_ref_pt.npy")

rest_dia = mm.numpy_to_geometry(
    contours_arr=dia_cont,
    catheter_arr=dia_cath,
    walls_arr=dia_wall,
    reference_arr=dia_ref,
)

rest_sys = mm.numpy_to_geometry(
    contours_arr=sys_cont,
    catheter_arr=sys_cath,
    walls_arr=sys_wall,
    reference_arr=sys_ref,
)

# Actual function call
rest = mm.from_array(
    mode="singlepair", 
    geometry_dia=rest_dia, 
    geometry_sys=rest_sys, 
    ouput_path="output/rest_array",
    steps_best_rotation=300,
    range_rotation_rad=1.57,
    interpolation_steps=0
)


Diastolic Geometry(17 contours), (17 catheter), Reference Point: Point(f=385, p=0, x=3.57, y=3.67, z=24.54, aortic=false) 
Systolic Geometry(17 contours), (17 catheter), Reference Point: Point(f=319, p=0, x=3.53, y=3.46, z=20.80, aortic=false)---------------------Finding optimal rotation ""/""---------------------


          Geometry A |           Geometry B |   Best Distance | Best Angle (°)
---------------------------------------------------------------------------
                     |                      |           0.389 |        0.000

---------------------Finding optimal rotation ""/""---------------------

          Geometry A |           Geometry B |   Best Distance | Best Angle (°)
---------------------------------------------------------------------------
                     |                      |           0.389 |        0.000

MESH .obj files: 2/2 written successfully
CATHETER .obj files: 2/2 written successfully
WALL .obj files: 2/2 written successfully
